### Setup

In [1]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
import cv2 as cv
import numpy as np
from datetime import datetime

from utils import get_coordinates, crop_and_mask_image

### Read the Video File

In [3]:
dir_project = "C:/Users/eats/projects/IASS"

name_video = "alpha"
path_video = f"asset/{name_video}.mp4"
path_video = os.path.join(dir_project, path_video)

cap = cv.VideoCapture(path_video)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (
    int(cap.get(x)) 
    for x in (cv.CAP_PROP_FRAME_WIDTH, cv.CAP_PROP_FRAME_HEIGHT, cv.CAP_PROP_FPS)
)


### Define Region Coordinates

In [15]:
success, imx = cap.read()

# # alpha.mp4
# region_points = [(780, 225), (1136, 622)]  # original
region_points = [(120, 30), (400, 475)]   # crop
x_min, x_max = 705, 1456
y_min, y_max = 132, 665
imx_crop = imx[y_min:y_max, x_min:x_max]
polygon = np.array([(0, 0), (388, 531), (750, 189), (304, 1)])

# # delta.mp4
# region_points = [(233, 800), (1795, 800)]  # original
# region_points = [(0, 250), (1178, 250)]    # crop
# x_min, x_max = 422, 1600
# y_min, y_max = 140, 510
# imx_crop = imx[y_min:y_max, x_min:x_max]
# polygon = np.array([(0, imx_crop.shape[0]), (273, 0), (950, 0), (imx_crop.shape[1], imx_crop.shape[0])])

# # beta.mp4
# region_points = [(275, 250), (580, 250)]  # original
# region_points = [(25, 130), (330, 130)]   # crop
# x_min, x_max = 250, 600
# y_min, y_max = 120, 290
# imx_crop = imx[y_min:y_max, x_min:x_max]
# polygon = np.array([(0, imx_crop.shape[0]), (130, 0), (245, 0), (imx_crop.shape[1], imx_crop.shape[0])])

# region_points = [(20, 400), (1080, 400), (1080, 360), (20, 360)]  # For rectangle region counting
# region_points = [(20, 400), (1080, 400), (1080, 360), (20, 360), (20, 400)]  # For polygon region counting

Original Image

In [10]:
success, imx = cap.read()

# Buat window dan set event callback
cv.namedWindow("Image", cv.WINDOW_NORMAL)
cv.namedWindow("Image")
cv.setMouseCallback("Image", get_coordinates)
cv.imshow("Image", imx)
cv.waitKey(0)

-1

Crop Image

In [16]:
success, imx = cap.read()

imx_crop = imx[y_min:y_max, x_min:x_max]
imx_crop = crop_and_mask_image(imx, x_min, x_max, y_min, y_max, polygon)
print(imx_crop.shape)

cv.namedWindow("Image", cv.WINDOW_NORMAL)
cv.namedWindow("Image")
cv.setMouseCallback("Image", get_coordinates)
cv.imshow("Image", imx_crop)
cv.waitKey(0)

(533, 751, 3)


-1

### Extract Each Frame of Video to JPG

In [18]:
i_skip = 1
name_datetime = str(datetime.now().date()).replace("-", "")
dir_output = os.path.join(dir_project, "asset", "dataset", f"{name_datetime}-{name_video}-skip_{i_skip}")
os.makedirs(dir_output, exist_ok=True)

cap = cv.VideoCapture(path_video)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (
    int(cap.get(x)) 
    for x in (cv.CAP_PROP_FRAME_WIDTH, cv.CAP_PROP_FRAME_HEIGHT, cv.CAP_PROP_FPS)
)

idx_img = 0
i = i_skip
cv.namedWindow("Video", cv.WINDOW_NORMAL)
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    
    if i == 0:
        i = i_skip
        continue
    
    # # imm = np.zeros_like(im0)
    # # imm[y_min:y_max, x_min:x_max] = im0[y_min:y_max, x_min:x_max]
    # imm = im0[y_min:y_max, x_min:x_max]
    
    # crop image
    imm = im0[y_min:y_max, x_min:x_max]

    # Create a black mask (same height & width as image, single channel)
    mask = np.zeros(imm.shape[:2], dtype=np.uint8)

    # Fill the polygon area with white (255)
    cv.fillPoly(mask, [polygon], color=255)

    # Apply the mask to keep only the polygon area
    imm = cv.bitwise_and(imm, imm, mask=mask)
    
    cv.imshow("Video", imm)
    if cv.waitKey(1) & 0xFF == ord("q"):
        break
    
    # video_writer.write(imm)
    
    path_output = os.path.join(dir_output, f'{name_video}_{idx_img}.jpg')
    cv.imwrite(path_output, imm)
    
    i -= 1
    idx_img += 1
    
    # break

cap.release()
# video_writer.release()
cv.destroyAllWindows()

Video frame is empty or video processing has been successfully completed.


In [ ]:
cv.waitKey(0)
cap.release()
cv.destroyAllWindows()